In [1]:
import gym
import numpy as np
import random
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [3]:
%cd drive/MyDrive/RML/KA/

/content/drive/MyDrive/RML/KA


In [ ]:
# Create the Taxi-v3 environment
env = gym.make("Taxi-v3", render_mode='ansi')

# Initialize the Q-table
num_states = env.observation_space.n
num_actions = env.action_space.n
q_table = np.zeros((num_states, num_actions))

# Define default hyperparameter values
default_hyperparameters = {
    "epsilon": 0.5,
    "alpha": 0.1,
    "gamma": 0.6,
    "max_epsilon": 1.0,
    "min_epsilon": 0.01,
    "decay_rate": 0.001
}

# Allow the user to input hyperparameters
hyperparameters = {}
for param, default_value in default_hyperparameters.items():
    user_input = input(f"Enter value for {param} (default: {default_value}): ")
    if user_input:
        hyperparameters[param] = float(user_input)
    else:
        hyperparameters[param] = default_value

epsilon = hyperparameters['epsilon'] # for saving

# Precompute all state-action pairs
state_action_pairs = [(s, a) for s in range(num_states) for a in range(num_actions)]

# Training loop
num_epochs = 10000
total_penalties = 0

start_time = time.time()

for epoch in range(num_epochs):
    state = env.reset()
    penalties = 0
    done = False

    while not done:
        # Epsilon-greedy exploration
        if random.uniform(0, 1) > hyperparameters["epsilon"]:
            action = np.argmax(q_table[state])
        else:
            action = env.action_space.sample()

        new_state, reward, done, _ = env.step(action)

        # Update Q-values
        old_value = q_table[state, action]
        next_max = np.max(q_table[new_state])
        new_value = (1 - hyperparameters["alpha"]) * old_value + \
                    hyperparameters["alpha"] * (reward + hyperparameters["gamma"] * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = new_state

    total_penalties += penalties

    # Decay epsilon
    hyperparameters["epsilon"] = max(hyperparameters["min_epsilon"],
                                      hyperparameters["epsilon"] * (1 - hyperparameters["decay_rate"]))

    # Print progress
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Total Penalties: {total_penalties}")

end_time = time.time()
training_time = end_time - start_time
print("Training completed in {:.2f} seconds.".format(training_time))

# Save Q-table to a file
np.save(f"q_table_{num_epochs}_{hyperparameters['epsilon']}_{training_time}_{total_penalties}.npy", q_table)